## Assignment Syntactic Analysis - HMMs and Viterbi algorithm for POS tagging

    As a part of this assignment, we will modify the Viterbi algorithm to solve the problem of unknown words using 
    few techniques.

### Goal for the Assignment -

    * Write the vanilla Viterbi algorithm for assigning POS tags (i.e. without dealing with unknown words).
    
    * Solve the problem of unknown words using at least two techniques. 
    
    * Compare the tagging accuracy after making these modifications with the vanilla Viterbi algorithm.
    

### Data Preparation

    For this assignment, we’ll use the Treebank dataset of NLTK with the 'universal' tagset. 
    The Universal tagset of NLTK comprises only 12 coarse tag classes as follows: Verb, Noun, Pronouns, 
    Adjectives, Adverbs, Adpositions, Conjunctions, Determiners, Cardinal Numbers, Particles, 
    Other/ Foreign words, Punctuations.

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# looking into few tagged sentences
print(nltk_data[:20])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Splitting into train and test
# sample size of 95:5
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('Reed', 'NOUN'), ('is', 'VERB'), ('paying', 'VERB'), ('an', 'DET'), ('interim', 'ADJ'), ('dividend', 'NOUN'), ('of', 'ADP'), ('4.6', 'NUM'), ('pence', 'NOUN'), (',', '.'), ('up', 'ADV'), ('15', 'NUM'), ('%', 'NOUN'), ('from', 'ADP'), ('4', 'NUM'), ('pence', 'NOUN'), ('a', 'DET'), ('year', 'NOUN'), ('earlier', 'ADJ'), ('.', '.')], [('The', 'DET'), ('finding', 'NOUN'), ('marks', 'VERB'), ('a', 'DET'), ('significant', 'ADJ'), ('step', 'NOUN'), ('in', 'ADP'), ('research', 'NOUN'), ('on', 'ADP'), ('``', '.'), ('bulk', 'NOUN'), ("''", '.'), ('superconductors', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-2', 'X'), ('are', 'VERB'), ('aimed', 'VERB'), ('*-1', 'X'), ('at', 'ADP'), ('use', 'NOUN'), ('in', 'ADP'), ('wires', 'NOUN'), ('for', 'ADP'), ('motors', 'NOUN'), (',', '.'), ('magnets', 'NOUN'), (',', '.'), ('generators', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('applications', 'NOUN'), ('.', '.')], [('They', 'PRON'), ('know', 'VERB'), ('that', 'ADP'), ('whenever', 'ADV'), ('

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95687

In [6]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Reed',
 'is',
 'paying',
 'an',
 'interim',
 'dividend',
 'of',
 '4.6',
 'pence',
 ',']

In [7]:
# vocabulary
V = set(tokens)
print(len(V))

12068


In [8]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [9]:
print(T)

{'ADV', 'PRT', 'ADJ', 'NUM', '.', 'PRON', 'VERB', 'ADP', 'NOUN', 'DET', 'X', 'CONJ'}


### POS Tagging Algorithm - HMM

### Emission Probabilities

In [10]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [11]:
w_given_t

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [13]:
# examples

# large
print("\n", "large")
print(word_given_tag('large', 'ADJ'))
print(word_given_tag('large', 'VERB'))
print(word_given_tag('large', 'NOUN'), "\n")

# will
print("\n", "will")
print(word_given_tag('will', 'VERB'))
print(word_given_tag('will', 'NOUN'))
print(word_given_tag('will', 'VERB'))

# book
print("\n", "book")
print(word_given_tag('book', 'NOUN'))
print(word_given_tag('book', 'VERB'))


 large
(29, 6087)
(0, 12869)
(0, 27513) 


 will
(262, 12869)
(1, 27513)
(262, 12869)

 book
(7, 27513)
(1, 12869)


### Transition Probabilities

In [14]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [15]:
# examples
print(t2_given_t1(t2='NOUN', t1='ADJ'))
print(t2_given_t1('NOUN', 'ADJ'))
print(t2_given_t1('NOUN', 'DET'))
print(t2_given_t1('NOUN', 'VERB'))
print(t2_given_t1(',', 'NOUN'))
print(t2_given_t1('ADP', 'ADP'))
print(t2_given_t1('VBG', 'NOUN'))

(4260, 6087)
(4260, 6087)
(5302, 8300)
(1425, 12869)
(0, 27513)
(161, 9387)
(0, 27513)


In [16]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [17]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [18]:
tags_df

,ADV,PRT,ADJ,NUM,.,PRON,VERB,ADP,NOUN,DET,X,CONJ
ADV,0.076382,0.014405,0.129313,0.031491,0.138358,0.014405,0.345729,0.118258,0.031826,0.070352,0.022446,0.007035
PRT,0.010104,0.001956,0.086375,0.057040,0.042699,0.017601,0.399609,0.019231,0.248696,0.101043,0.013364,0.002282
ADJ,0.004764,0.010514,0.067685,0.021028,0.064400,0.000657,0.011828,0.077378,0.699852,0.004600,0.020207,0.017086
NUM,0.002978,0.027099,0.033353,0.185527,0.119119,0.001191,0.017868,0.035140,0.351697,0.003276,0.209053,0.013699
.,0.052542,0.002339,0.044894,0.079982,0.092578,0.065677,0.089069,0.091408,0.221682,0.174899,0.027530,0.057310
PRON,0.034911,0.012413,0.073701,0.007370,0.039953,0.006982,0.484872,0.022110,0.210628,0.009697,0.092708,0.004655
VERB,0.081047,0.031238,0.064885,0.022768,0.035045,0.035045,0.169244,0.092004,0.110731,0.134121,0.218510,0.005362
ADP,0.013849,0.001385,0.104932,0.062853,0.039949,0.068925,0.008203,0.017151,0.323213,0.323426,0.035368,0.000746
NOUN,0.016901,0.044343,0.011994,0.009196,0.239123,0.004652,0.146440,0.177262,0.265111,0.013157,0.029114,0.042707
DET,0.012530,0.000241,0.206506,0.022289,0.016627,0.003614,0.038675,0.009398,0.638795,0.005181,0.045663,0.000482


## PART 1 - Unmodified Vanilla Viterbi based POS tagger

In [19]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



### Evaluating on Validation Set (5% of entire dataset)

In [35]:
test_set

[[('Esso', 'NOUN'),
  ('Australia', 'NOUN'),
  ('Ltd.', 'NOUN'),
  (',', '.'),
  ('a', 'DET'),
  ('unit', 'NOUN'),
  ('of', 'ADP'),
  ('New', 'ADJ'),
  ('York-based', 'ADJ'),
  ('Exxon', 'NOUN'),
  ('Corp.', 'NOUN'),
  (',', '.'),
  ('and', 'CONJ'),
  ('Broken', 'NOUN'),
  ('Hill', 'NOUN'),
  ('Pty.', 'NOUN'),
  ('operate', 'VERB'),
  ('the', 'DET'),
  ('fields', 'NOUN'),
  ('in', 'ADP'),
  ('a', 'DET'),
  ('joint', 'ADJ'),
  ('venture', 'NOUN'),
  ('.', '.')],
 [('``', '.'),
  ('New', 'ADJ'),
  ('managers', 'NOUN'),
  ('would', 'VERB'),
  ('think', 'VERB'),
  ('a', 'DET'),
  ('little', 'ADV'),
  ('more', 'ADJ'),
  ('like', 'ADP'),
  ('Wall', 'NOUN'),
  ('Street', 'NOUN'),
  (',', '.'),
  ("''", '.'),
  ('Mr.', 'NOUN'),
  ('McMillin', 'NOUN'),
  ('added', 'VERB'),
  ('*T*-1', 'X'),
  ('.', '.')],
 [('*-2', 'X'),
  ('Unable', 'ADJ'),
  ('*-3', 'X'),
  ('to', 'PRT'),
  ('unload', 'VERB'),
  ('UAL', 'NOUN'),
  ('and', 'CONJ'),
  ('other', 'ADJ'),
  ('airline', 'NOUN'),
  ('shares', 'NOUN'

In [159]:
test_run_base =[tup for sent in test_set for tup in sent]

[('Esso', 'NOUN'),
 ('Australia', 'NOUN'),
 ('Ltd.', 'NOUN'),
 (',', '.'),
 ('a', 'DET'),
 ('unit', 'NOUN'),
 ('of', 'ADP'),
 ('New', 'ADJ'),
 ('York-based', 'ADJ'),
 ('Exxon', 'NOUN'),
 ('Corp.', 'NOUN'),
 (',', '.'),
 ('and', 'CONJ'),
 ('Broken', 'NOUN'),
 ('Hill', 'NOUN'),
 ('Pty.', 'NOUN'),
 ('operate', 'VERB'),
 ('the', 'DET'),
 ('fields', 'NOUN'),
 ('in', 'ADP'),
 ('a', 'DET'),
 ('joint', 'ADJ'),
 ('venture', 'NOUN'),
 ('.', '.'),
 ('``', '.'),
 ('New', 'ADJ'),
 ('managers', 'NOUN'),
 ('would', 'VERB'),
 ('think', 'VERB'),
 ('a', 'DET'),
 ('little', 'ADV'),
 ('more', 'ADJ'),
 ('like', 'ADP'),
 ('Wall', 'NOUN'),
 ('Street', 'NOUN'),
 (',', '.'),
 ("''", '.'),
 ('Mr.', 'NOUN'),
 ('McMillin', 'NOUN'),
 ('added', 'VERB'),
 ('*T*-1', 'X'),
 ('.', '.'),
 ('*-2', 'X'),
 ('Unable', 'ADJ'),
 ('*-3', 'X'),
 ('to', 'PRT'),
 ('unload', 'VERB'),
 ('UAL', 'NOUN'),
 ('and', 'CONJ'),
 ('other', 'ADJ'),
 ('airline', 'NOUN'),
 ('shares', 'NOUN'),
 (',', '.'),
 ('takeover-stock', 'ADJ'),
 ('specula

In [37]:
# testing our Viterbi algorithm on test dataset

# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]
# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_set

[[('Esso', 'NOUN'),
  ('Australia', 'NOUN'),
  ('Ltd.', 'NOUN'),
  (',', '.'),
  ('a', 'DET'),
  ('unit', 'NOUN'),
  ('of', 'ADP'),
  ('New', 'ADJ'),
  ('York-based', 'ADJ'),
  ('Exxon', 'NOUN'),
  ('Corp.', 'NOUN'),
  (',', '.'),
  ('and', 'CONJ'),
  ('Broken', 'NOUN'),
  ('Hill', 'NOUN'),
  ('Pty.', 'NOUN'),
  ('operate', 'VERB'),
  ('the', 'DET'),
  ('fields', 'NOUN'),
  ('in', 'ADP'),
  ('a', 'DET'),
  ('joint', 'ADJ'),
  ('venture', 'NOUN'),
  ('.', '.')],
 [('``', '.'),
  ('New', 'ADJ'),
  ('managers', 'NOUN'),
  ('would', 'VERB'),
  ('think', 'VERB'),
  ('a', 'DET'),
  ('little', 'ADV'),
  ('more', 'ADJ'),
  ('like', 'ADP'),
  ('Wall', 'NOUN'),
  ('Street', 'NOUN'),
  (',', '.'),
  ("''", '.'),
  ('Mr.', 'NOUN'),
  ('McMillin', 'NOUN'),
  ('added', 'VERB'),
  ('*T*-1', 'X'),
  ('.', '.')],
 [('*-2', 'X'),
  ('Unable', 'ADJ'),
  ('*-3', 'X'),
  ('to', 'PRT'),
  ('unload', 'VERB'),
  ('UAL', 'NOUN'),
  ('and', 'CONJ'),
  ('other', 'ADJ'),
  ('airline', 'NOUN'),
  ('shares', 'NOUN'

In [38]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [39]:
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  787.3224277496338
[('Esso', 'NOUN'), ('Australia', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('a', 'DET'), ('unit', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York-based', 'NOUN'), ('Exxon', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Broken', 'ADV'), ('Hill', 'NOUN'), ('Pty.', 'ADV'), ('operate', 'VERB'), ('the', 'DET'), ('fields', 'NOUN'), ('in', 'ADP'), ('a', 'DET'), ('joint', 'ADJ'), ('venture', 'NOUN'), ('.', '.'), ('``', '.'), ('New', 'NOUN'), ('managers', 'NOUN'), ('would', 'VERB'), ('think', 'VERB'), ('a', 'DET'), ('little', 'ADJ'), ('more', 'ADJ'), ('like', 'ADP'), ('Wall', 'NOUN'), ('Street', 'NOUN'), (',', '.'), ("''", '.'), ('Mr.', 'NOUN'), ('McMillin', 'NOUN'), ('added', 'VERB'), ('*T*-1', 'X'), ('.', '.'), ('*-2', 'X'), ('Unable', 'ADV'), ('*-3', 'X'), ('to', 'PRT'), ('unload', 'ADV'), ('UAL', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('airline', 'NOUN'), ('shares', 'NOUN'), (',', '.'), ('takeover-stock', 'ADV'), ('speculators', 'NOUN

In [40]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [41]:
accuracy = len(check)/len(tagged_seq)
accuracy

0.9027861294848667

##### Lets look into incorrectly tagged words

In [42]:
#incorrect tag
incorr = [i for i, j in zip(tagged_seq, test_run_base) if i != j] 

In [45]:
len(incorr)

485

In [44]:
from collections import Counter
tags = [pair[1] for pair in incorr]
tag_counts = Counter(tags)
tag_counts

Counter({'ADJ': 31,
         'ADP': 30,
         'ADV': 358,
         'DET': 7,
         'NOUN': 26,
         'NUM': 1,
         'PRT': 4,
         'VERB': 28})

### Evaluating on Test Set (Unknown data)

In [34]:
with open('Test_sentences.txt') as val_set:
    for line in val_set:
        sentence_test = line.strip()
        print ("--------------------------------------------------")
        words = word_tokenize(sentence_test)
        print (sentence_test)
        start = time.time()
        tagged_seq_val = Viterbi(words)
        print(tagged_seq_val)
        print ("--------------------------------------------------")
        end = time.time()
        difference = end-start        

--------------------------------------------------
Android is a mobile operating system developed by Google.
[('Android', 'ADV'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'ADV'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
[('Android', 'ADV'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADV'), ('OS', 'ADV'), ('worldwide', 'ADV'), ('on', 'ADP'), ('smartphones', 'ADV'), ('since', 'ADP'), ('2011', 'ADV'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'ADV'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
[('Google',

### Observations (Unmodified Vanilla Viterbi) based on test and validation dataset :-

    1. A Normal Viterbi algorithm with Universal tagset is showing ~90% accuracy.
    2. Out of 485 wrongly tagged words; we see 358 are ADV - thats because the 1st tag in our universal POS is ADV.
    3. Some of the notable tag types getting mis-identified:-
        
        * Unknown Proper nouns like - 'Android', 'Google'
        * Hyphenated words like     - 'best-selling', 'multibillion-dollar', *-3, T*-1
        * Numbers/decimals like     - '24.95', '2018'

## PART 2 - Solving the problem of unknown words -

    Now we will try few approaches to modify the Viterbi algorithm to solve the problem of unknown 
    words/incorrect tags.

### Approach 1 - Viterbi + Rule Based tagging

    For the pattern types that are most likely getting wrongly tagged, we will used regex to fix the problem 
    in this approach.

##### Regex Rule based

In [250]:
# specify patterns for rule based tagging
patterns = [
    (r'.*ing$', 'VERB'),                        # continuous verbs
    (r'.*ed$', 'VERB'),                         # past tense verbs
    (r'.*es$', 'VERB'),                         # 3rd singular present
    (r'.*ould$', 'VERB'),                       # modals
    (r'(ate\b|fy\b|ize\b|\ben|\bem)', 'VERB'),  # VERBS
    (r'.*\'s$', 'NOUN'),                        # possessive nouns
    (r'.*s$', 'NOUN'),                          # plural nouns
    (r'^.*-\d+$', 'X'),                         # hyphenated word ending with number        
    (r'^[0-9]*(\.[0-9]{,2})?$', 'NUM'),         # cardinal numbers
    (r'\w+(?:-\w+)+', 'ADJ'),                   # hyphenated words
    (r'(\bun|\bin|ble\b|ry\b|ish\b|ious\b|ical\b|\bnon)', 'ADJ'),  # Adjectives
    (r'.*', 'NOUN')                             # nouns as default
]

In [251]:
#creating regex pattern
regexp_tagger = nltk.RegexpTagger(patterns)

### Note on modification-1

        By Viterbi Heuristic, the algorithm will choose the tag having max (emission_p * transition_p) probability.
        However, there are situations where algorithm can not predict the tag because the max probability is 0.
        In those scenarios we are going to route the alogorithm to use "regexp_tagger" i.e. rule based tagging.

In [252]:
# Viterbi Heuristic - Modified 1
def ViterbiMod1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        #modification to route to regex tagger in case of pmax is zero
        if pmax == 0.0:
            dummyp = []
            dummyp.append(word)
            [state.append(x[1]) for x in regexp_tagger.tag(dummyp)]          
        # getting state for which probability is maximum
        else:
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

### Evaluating on Validation Set (5% of entire dataset)
    -- Time taken in seconds:  838 secs

In [253]:
# tagging the test sentences
start = time.time()
tagged_seq2 = ViterbiMod1(test_tagged_words)
end = time.time()
difference = end-start

In [254]:
print("Time taken in seconds: ", difference)
print(tagged_seq2)
#print(test_run_base)

Time taken in seconds:  838.9062337875366


[('Esso', 'NOUN'), ('Australia', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('a', 'DET'), ('unit', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York-based', 'NOUN'), ('Exxon', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Broken', 'NOUN'), ('Hill', 'NOUN'), ('Pty.', 'NOUN'), ('operate', 'VERB'), ('the', 'DET'), ('fields', 'NOUN'), ('in', 'ADP'), ('a', 'DET'), ('joint', 'ADJ'), ('venture', 'NOUN'), ('.', '.'), ('``', '.'), ('New', 'NOUN'), ('managers', 'NOUN'), ('would', 'VERB'), ('think', 'VERB'), ('a', 'DET'), ('little', 'ADJ'), ('more', 'ADJ'), ('like', 'ADP'), ('Wall', 'NOUN'), ('Street', 'NOUN'), (',', '.'), ("''", '.'), ('Mr.', 'NOUN'), ('McMillin', 'NOUN'), ('added', 'VERB'), ('*T*-1', 'X'), ('.', '.'), ('*-2', 'X'), ('Unable', 'NOUN'), ('*-3', 'X'), ('to', 'PRT'), ('unload', 'ADJ'), ('UAL', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('airline', 'NOUN'), ('shares', 'NOUN'), (',', '.'), ('takeover-stock', 'ADJ'), ('speculators', 'NOUN'), (',', '.'), ('or', 'CONJ'), ('risk'

#### Evaluating tagging accuracy

In [255]:
# accuracy
check2 = [i for i, j in zip(tagged_seq2, test_run_base) if i == j] 

In [256]:
accuracy2 = len(check2)/len(tagged_seq2)
accuracy2

0.9526959310483063

### Evaluating on Test Set (Unknown data)

In [53]:
with open('Test_sentences.txt') as val_set:
    for line in val_set:
        sentence_test = line.strip()
        print ("--------------------------------------------------")
        words = word_tokenize(sentence_test)
        print (sentence_test)
        start = time.time()
        tagged_seq_val2 = ViterbiMod1(words)
        print(tagged_seq_val2)
        print ("--------------------------------------------------")
        end = time.time()
        difference = end-start        

--------------------------------------------------
Android is a mobile operating system developed by Google.
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'VERB'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
[('G

### Observations (Vanilla Viterbi + Rule based) based on test and validation dataset :-

    1. Vanilla Viterbi + Rule based algorithm with Universal tagset is showing ~95% accuracy, 
    which is quite substantial improvements to naive Vanilla Viterbi.
    2. Previously mis-identified tags gettings properly identified:-
        
        * Unknown proper nouns like 'Android', 'Google' are correctly tagged as 'NOUN'
        * Hyphenated words like 'best-selling', 'multibillion-dollar' are correctly tagged as 'ADJ'
        * Numeric hyphenated words like *-3, T*-1 are correctly tagged as 'X'
        * Numbers/decimals like '24.95', '2018', 1,500 are correctly tagged as 'NUM'

### Approach 2 - Viterbi + Lexicon + Rule Based tagging

    For the words having max probability as 0; we will pass through Lexicon and use rule based as "back-off"

### Note on modification-2

        By Viterbi Heuristic, the algorithm will choose the tag having max (emission_p * transition_p) probability.
        However, there are situations where algorithm can not predict the correct tag because max probability is 0.
        In those scenarios we are going to route the alogorithm to use Lexicon based tagging with 
        rule based as "backoff".

In [58]:
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=regexp_tagger)

In [57]:
# Viterbi Heuristic Modification 2
def ViterbiMod2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        #modification to route to lexicon based/rule based
        if pmax == 0.0:
            dummyp = []
            dummyp.append(word)
            [state.append(x[1]) for x in lexicon_tagger.tag(dummyp)]          
        # getting state for which probability is maximum
        else:
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

### Evaluating on Validation Set (5% of entire dataset)

In [59]:
# tagging the test sentences
start = time.time()
tagged_seq3 = ViterbiMod2(test_tagged_words)
end = time.time()
difference = end-start

In [60]:
print("Time taken in seconds: ", difference)
print(tagged_seq3)

Time taken in seconds:  697.5415170192719
[('Esso', 'NOUN'), ('Australia', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('a', 'DET'), ('unit', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York-based', 'NOUN'), ('Exxon', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Broken', 'NOUN'), ('Hill', 'NOUN'), ('Pty.', 'NOUN'), ('operate', 'VERB'), ('the', 'DET'), ('fields', 'NOUN'), ('in', 'ADP'), ('a', 'DET'), ('joint', 'ADJ'), ('venture', 'NOUN'), ('.', '.'), ('``', '.'), ('New', 'NOUN'), ('managers', 'NOUN'), ('would', 'VERB'), ('think', 'VERB'), ('a', 'DET'), ('little', 'ADJ'), ('more', 'ADJ'), ('like', 'ADP'), ('Wall', 'NOUN'), ('Street', 'NOUN'), (',', '.'), ("''", '.'), ('Mr.', 'NOUN'), ('McMillin', 'NOUN'), ('added', 'VERB'), ('*T*-1', 'X'), ('.', '.'), ('*-2', 'X'), ('Unable', 'NOUN'), ('*-3', 'X'), ('to', 'PRT'), ('unload', 'NOUN'), ('UAL', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('airline', 'NOUN'), ('shares', 'NOUN'), (',', '.'), ('takeover-stock', 'ADJ'), ('speculators', '

#### Evaluating tagging accuracy

In [61]:
# accuracy
check3 = [i for i, j in zip(tagged_seq3, test_run_base) if i == j] 

In [64]:
accuracy3 = len(check3)/len(tagged_seq3)
accuracy3

0.952495490078172

### Evaluating on Test Set (Unknown data)

In [63]:
with open('Test_sentences.txt') as val_set:
    for line in val_set:
        sentence_test = line.strip()
        print ("--------------------------------------------------")
        words = word_tokenize(sentence_test)
        print (sentence_test)
        start = time.time()
        tagged_seq_val3 = ViterbiMod2(words)
        print(tagged_seq_val3)
        print ("--------------------------------------------------")
        end = time.time()
        difference = end-start        

--------------------------------------------------
Android is a mobile operating system developed by Google.
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'VERB'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
[('G

### Observations (Vanilla Viterbi + Lexicon + Rule based) based on test and validation dataset :-

    1. Vanilla Viterbi + Lexicon + Rule based algorithm with Universal tagset is showing ~95% accuracy, 
    quite similar to approach 2.
    2. Tags mis-identified by normal Viterbi is gettings properly identified:-
        
        * Unknown proper nouns like 'Android', 'Google', 'FIFA' are correctly tagged as 'NOUN'
        * Hyphenated words like 'best-selling', 'multibillion-dollar' are correctly tagged as 'ADJ'
        * Numeric hyphenated words like *-3, T*-1 are correctly tagged as 'X'
        * Numbers/decimals like '24.95', '2018', 1,500 are correctly tagged as 'NUM'

### Approach 3 - Viterbi + nltk.pos_tag

    For the pattern types that are most likely getting wrongly tagged, we will used nltk.pos_tag to fix the problem 
    in this approach.

### Note on modification-3

        By Viterbi Heuristic, the algorithm will choose the tag having max (emission_p * transition_p) probability.
        However, there are situations where algorithm can not predict the tag because of max probability being 0.
        In those scenarios we are going to route the alogorithm to use nltk.pos_tag (universal tagset) 
        to fill the errors.

In [65]:
# Viterbi Heuristic
def ViterbiMod3(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        #modified to use nltk.pos_tag
        if pmax == 0.0:
            dummyp = []
            dummyp.append(word)
            [state.append(x[1]) for x in nltk.pos_tag(dummyp,tagset='universal')]          
        # getting state for which probability is maximum
        else:
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

### Evaluating on Validation Set (5% of entire dataset)

In [66]:
# tagging the test sentences
start = time.time()
tagged_seq4 = ViterbiMod3(test_tagged_words)
end = time.time()
difference = end-start

In [71]:
print("Time taken in seconds: ", difference)
print(tagged_seq4)
#print(test_run_base)

Time taken in seconds:  0.0
[('Esso', 'NOUN'), ('Australia', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('a', 'DET'), ('unit', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York-based', 'NOUN'), ('Exxon', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Broken', 'NOUN'), ('Hill', 'NOUN'), ('Pty.', 'NOUN'), ('operate', 'VERB'), ('the', 'DET'), ('fields', 'NOUN'), ('in', 'ADP'), ('a', 'DET'), ('joint', 'ADJ'), ('venture', 'NOUN'), ('.', '.'), ('``', '.'), ('New', 'NOUN'), ('managers', 'NOUN'), ('would', 'VERB'), ('think', 'VERB'), ('a', 'DET'), ('little', 'ADJ'), ('more', 'ADJ'), ('like', 'ADP'), ('Wall', 'NOUN'), ('Street', 'NOUN'), (',', '.'), ("''", '.'), ('Mr.', 'NOUN'), ('McMillin', 'NOUN'), ('added', 'VERB'), ('*T*-1', 'X'), ('.', '.'), ('*-2', 'X'), ('Unable', 'ADJ'), ('*-3', 'X'), ('to', 'PRT'), ('unload', 'NOUN'), ('UAL', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('airline', 'NOUN'), ('shares', 'NOUN'), (',', '.'), ('takeover-stock', 'NOUN'), ('speculators', 'NOUN'), (',', 

##### Evaluating testing accuracy

In [68]:
# accuracy
check4 = [i for i, j in zip(tagged_seq4, test_run_base) if i == j] 

In [69]:
accuracy4 = len(check4)/len(tagged_seq4)
accuracy4

0.9542994588093806

### Evaluating on Test Set (Unknown data)

In [70]:
with open('Test_sentences.txt') as val_set:
    for line in val_set:
        sentence_test = line.strip()
        print ("--------------------------------------------------")
        words = word_tokenize(sentence_test)
        print (sentence_test)
        start = time.time()
        tagged_seq_val4 = ViterbiMod3(words)
        print(tagged_seq_val4)
        print ("--------------------------------------------------")
        end = time.time()
        difference = end-start        

--------------------------------------------------
Android is a mobile operating system developed by Google.
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'NOUN'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
[('G

### Observations (Viterbi + nltk.pos_tag) based on test and validation dataset :-

    1. Viterbi + nltk.pos_tag based algorithm with Universal tagset is also showing ~95% accuracy, 
    which is 5% more than the normal viterbi algorithm.
    2. Tags mis-identified by Viterbi is gettings properly identified:-
        
        * Unknown proper nouns like 'Android', 'Google', 'FIFA' are correctly tagged as 'NOUN'
        * Hyphenated words like 'best-selling', 'multibillion-dollar' are correctly tagged as 'ADJ'
        * Numeric hyphenated words like *-3, T*-1 are correctly tagged as 'X'
        * Numbers/decimals like '24.95', '2018', 1,500 are correctly tagged as 'NUM'

### Approach 4 - Viterbi + DecisionTree
    
    This is a more elaborate approach; where we use sklearn DecisionTree to correct the errornous tags by Viterbi.
    For the words unpredictable by Viterbi; we will pass through DecisionTree to get corrected tag.

#### Training POS Tagger using scikit-learn

In [203]:
from nltk import word_tokenize, pos_tag
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

In [198]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [199]:
#strip the tags from tagged corpus 
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [200]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y

**Warning - training takes a very long time and memory!!**

In [240]:
X, y = transform_to_dataset(train_set)
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:40000], y[:40000])

Pipeline(memory=None,
     steps=[('vectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('classifier', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

#### Creating pos_tag function to predict tags

In [241]:
#test function call 
def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return list(tags)

### Note on modification-4

        By Viterbi Heuristic, the algorithm will choose the tag having max (emission_p * transition_p) probability.
        However, there are situations where algorithm can not predict the tag because the max probability is 0.
        In those scenarios we are going to route the alogorithm to use our self created 
        pos_tag (DecisionTreeClassifier) to update the unseen words.

In [244]:
# Viterbi Heuristic
def ViterbiMod4(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        #modified to use nltk.pos_tag
        if pmax == 0.0:
            dummyp = []
            dummyp.append(word)
            [state.append(x) for x in pos_tag(dummyp)]
        # getting state for which probability is maximum
        else:
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

### Evaluating on Validation Set (5% of entire dataset)

In [245]:
# tagging the test sentences
start = time.time()
tagged_seq5 = ViterbiMod4(test_tagged_words)
end = time.time()
difference = end-start

In [246]:
print("Time taken in seconds: ", difference)
print(tagged_seq5)
#print(test_run_base)

Time taken in seconds:  833.0606453418732


[('Esso', 'NOUN'), ('Australia', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('a', 'DET'), ('unit', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('York-based', 'NOUN'), ('Exxon', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Broken', 'NOUN'), ('Hill', 'NOUN'), ('Pty.', 'NOUN'), ('operate', 'VERB'), ('the', 'DET'), ('fields', 'NOUN'), ('in', 'ADP'), ('a', 'DET'), ('joint', 'ADJ'), ('venture', 'NOUN'), ('.', '.'), ('``', '.'), ('New', 'NOUN'), ('managers', 'NOUN'), ('would', 'VERB'), ('think', 'VERB'), ('a', 'DET'), ('little', 'ADJ'), ('more', 'ADJ'), ('like', 'ADP'), ('Wall', 'NOUN'), ('Street', 'NOUN'), (',', '.'), ("''", '.'), ('Mr.', 'NOUN'), ('McMillin', 'NOUN'), ('added', 'VERB'), ('*T*-1', 'X'), ('.', '.'), ('*-2', 'X'), ('Unable', 'ADP'), ('*-3', 'X'), ('to', 'PRT'), ('unload', 'VERB'), ('UAL', 'NOUN'), ('and', 'CONJ'), ('other', 'ADJ'), ('airline', 'NOUN'), ('shares', 'NOUN'), (',', '.'), ('takeover-stock', 'ADJ'), ('speculators', 'NOUN'), (',', '.'), ('or', 'CONJ'), ('risk'

In [247]:
# accuracy
check5 = [i for i, j in zip(tagged_seq5, test_run_base) if i == j] 

In [248]:
accuracy5 = len(check5)/len(tagged_seq5)
accuracy5

0.9534976949288435

### Evaluating on Test Set (Unknown data)

In [249]:
with open('Test_sentences.txt') as val_set:
    for line in val_set:
        sentence_test = line.strip()
        print ("--------------------------------------------------")
        words = word_tokenize(sentence_test)
        print (sentence_test)
        start = time.time()
        tagged_seq_val5 = ViterbiMod4(words)
        print(tagged_seq_val5)
        print ("--------------------------------------------------")
        end = time.time()
        difference = end-start        

--------------------------------------------------
Android is a mobile operating system developed by Google.
[('Android', 'CONJ'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
[('Android', 'CONJ'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'ADJ'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'X'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'X'), ('.', '.')]
--------------------------------------------------
--------------------------------------------------
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
[('Google'

### Observations (Viterbi + Decesion Tree) based on test and validation dataset :-

    1. Viterbi + pos_tag (DecisionTreeClassifier) based algorithm with Universal tagset is also showing ~95% accuracy, 
    however computationally more expensive than other approaches.
    2. Tags mis-identified by normal Viterbi is gettings properly identified:-
        
        * Unknown proper nouns like 'Android', 'Google', 'FIFA' are correctly tagged as 'NOUN'
        * Hyphenated words like 'best-selling', 'multibillion-dollar' are correctly tagged as 'ADJ'
        * Numeric hyphenated words like *-3, T*-1 are correctly tagged as 'X'
        * Numbers/decimals like '24.95', '2018', 1,500 are correctly tagged as 'NUM'

## Inferences and Observations -


In the given assignment, I have use 4 different approaches:-
    
   
   **Approach 1 - Viterbi + Rule Based tagging**
   
   **Approach 2 - Viterbi + Lexicon + Rule Based tagging**
   
   **Approach 3 - Viterbi + nltk.pos_tag**
   
   **Approach 4 - Viterbi + DecisionTree**

    1. Normal Viterbi with universal tagger gives ~90% accuracy; however modified version gives ~95% accuracy
        which in 5% higher.
    
    2. Tags mis-identified by Viterbi is gettings properly identified:-
        
        * Unknown proper nouns like 'Android', 'Google', 'FIFA', 'Twitter' are correctly tagged as 'NOUN'
        * Hyphenated words like 'best-selling', 'multibillion-dollar' are correctly tagged as 'ADJ'
        * Numeric hyphenated words like *-3, T*-1 are correctly tagged as 'X'
        * Numbers/decimals like '24.95', '2018', 1,500 are correctly tagged as 'NUM'
     
     3. Viterbi + DecisionTree based algorithm is computationally more expensive than rest of the approaches.
     
     4. All these algorithms have been used in tandom with Viterbi.
         Only in the occassion when Viterbi is unable to add tag to words (i.e. all the tags show 0.0 probability), 
         the algorithm has been routed to use the approches.

    **************************************************************************************************